# Переменные:

In [1]:
#Номер текущей недели
week_number = 47

#Номер текущего года
year_number = 2021

#В какие недели 2021 года был локдаун
weeks_lockdown_2021 = [44, 45, 46]

#На сколько недель нужен прогноз?
horizon = 12

#Размер тестовой выборки (на случай, если нужна валидация) - в неделях (шт)
test_size=0

In [2]:
import pandas as pd
import pyodbc
import adodbapi as ado
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings; warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults

import itertools

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (13,10)

import seaborn as sns
from sklearn.model_selection import learning_curve 
from sklearn.metrics import make_scorer 
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import TweedieRegressor
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

from statsmodels.tsa.filters.filtertools import convolution_filter
from statsmodels.tsa.api import ExponentialSmoothing 

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV 

# Вспомогательные функции:

In [3]:
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

# Загрузка и предобработка данных

In [4]:
%%time
# Запрос к кубу

conn_args = {'timeout' : 90000}
with ado.connect('''private data''',  conn_args) as con:
 
        with con.cursor() as cur:
            first_mdx = '''   
             SELECT NON EMPTY 
             { 
             [Measures].[Продажи руб Без НДС]
             } 
             
             ON COLUMNS, NON EMPTY 
             { 
             ([Date].[Year For Weeks].[Year For Weeks].ALLMEMBERS * 
             [Date].[Month For Weeks].[Month For Weeks].ALLMEMBERS * 
             [Date].[Week].[Week].ALLMEMBERS *  
             [Модели].[Новые ЦГ 0-8].[Новые ЦГ 0-8].ALLMEMBERS * 
             [Модели].[ТГ плюс].[ТГ плюс].ALLMEMBERS ) 
             } 
             
             DIMENSION PROPERTIES MEMBER_CAPTION, MEMBER_UNIQUE_NAME 
             ON ROWS FROM ( SELECT ( [Date].[Year].&[2017] : [Date].[Year].&[2021] ) 
             ON COLUMNS FROM ( SELECT ( { [Магазины].[Канал продаж].&[OffLine] } ) 
             ON COLUMNS FROM [SalesChecksCustomers])) 
             
             WHERE ( [Магазины].[Канал продаж].&[OffLine], [Модели].[БиА].&[Main] ) 
            '''


            
            # all model

            cur.execute(first_mdx)
            data0 = cur.fetchall()
            ar = np.array(data0.ado_results)
            df = pd.DataFrame(ar).transpose()
            df.columns = data0.columnNames.keys()

            
df = df[[col for col in df.columns if 'unique' not in col]]

Wall time: 32.7 s


In [5]:
#Создаем копию выгруженых данных

data = df.copy()

In [6]:
#Переименование колонок и перекодировка номера недели и года в int, Sales to float

data.columns = ['year','month', 'week', 'TG', 'ТГ+', 'Sales']

data['month']=data['month'].str.lower()
data['week']=data.week.str.split(expand = True)[0].astype(int)
data['year']=data.year.str.split(expand = True)[0].astype(int)
data['Sales']=data.Sales.str.split(expand = True)[0].astype(float)

In [7]:
#drop current week

data = data.loc[~((data.year == year_number)&(data.week == week_number))]

In [8]:
#drop Boys & Girls 0-1

data = data[(data.TG!='G0-1')&(data.TG!='B0-1')]
data = data.reset_index(drop=True)

In [9]:
#column with uniques TG & dep+

data_for_final = data.copy()

data['ТГ+'] = data['ТГ+'].str.replace(" " , "")
data['ТГ+'] = data['ТГ+'].str.replace("-" , "")
data['TG'] = data['TG'].str.replace("+" , "")
data['TG'] = data['TG'].str.replace("-" , "")
data['uni'] = data['TG']+ data['ТГ+']

data = data.drop(['TG', 'ТГ+'], axis=1)

data_for_final['uni'] = data.uni

In [10]:
#Удаляем категории, сожержащие нули

# drop_categories = data[(data.Sales<1)|(data.year)].uni.unique()
group_data = data.groupby('uni')['year'].count().reset_index()
is_na_data = group_data[group_data.year != group_data.year.max()].uni.unique()

# data = data.loc[~( (data.uni.isin(drop_categories) ) | (data.uni.isin(is_na_data) )) ]

data = data.loc[~( (data.uni.isin(is_na_data) )) ]

In [11]:
#list of categories

forecast_categories = data.uni.unique()

In [12]:
#Словарь со всеми категориями

frames_dict = {'%s' % x : data[data.uni==x] for x in forecast_categories}

In [13]:
#Фильтрация всех категорий

for i in forecast_categories:

    frames_dict[i] = frames_dict[i][(frames_dict[i]['week']!=53)&
                                    (frames_dict[i]['year']>=2017)&
                                    (frames_dict[i]['year']<=2021)].reset_index(drop=True)
    

# Создание подписей к графикам

In [14]:
sample_data=frames_dict['G13JEANSCOMMERCIAL']

In [15]:
#Размер тренирововчной выборки

size = len(sample_data)-test_size

In [16]:
full_log_label = int(len(sample_data)/5)
train_log_label = int(size/5)

if test_size >=5:
    test_log_label = int(test_size/5)
else:
    test_log_label = int(test_size/5)+1

In [17]:
lable_data = sample_data.copy()
lable_data['label'] = lable_data[['week', 'year']].apply(tuple, axis=1)

In [18]:
#Подписи к целым графикам

full_labels =[]
full_labels.append((0,0))
full_labels.append(lable_data.label[0])
full_labels.append(lable_data.label[full_log_label-1:len(lable_data):full_log_label].values.tolist())
full_labels=flatten_list(full_labels)

In [19]:
#Подписи к тренировочным графикам

train_data = lable_data[:size].reset_index(drop=True)

train_labels =[]
train_labels.append((0,0))
train_labels.append(train_data.label[0])
train_labels.append(train_data.label[train_log_label-1:len(train_data):train_log_label].values.tolist())
train_labels=flatten_list(train_labels)

In [20]:
#Подписи к валидационным графикам

if test_size != 0:
    val_data = lable_data[size:].reset_index(drop=True)

    val_labels =[]
    val_labels.append((0,0))
    val_labels.append(val_data.label[0])
    val_labels.append(val_data.label[test_log_label-1:len(val_data):test_log_label].values.tolist())
    val_labels=flatten_list(val_labels)

# График фактических продаж

In [21]:
# for i in forecast_categories:
#     plt.title(i)
#     plt.plot(frames_dict[i].Sales)
#     plt.show()

# Корректировка инфляции

In [22]:
#Данные ИПЦ

ipc = pd.read_excel('ipc2.xlsx')
ipc.head()

,year,month,i,Unnamed: 3
0,2016,январь,100.96,NaN
1,2016,февраль,100.63,NaN
2,2016,март,100.46,NaN
3,2016,апрель,100.44,NaN
4,2016,май,100.41,NaN


In [23]:
#Расчет множителя ИПЦ

ipc = ipc[['year', 'month', 'i']]
ipc=ipc.iloc[:-3,:]
ipc['CPI_Multiplier'] = ipc['i'].iloc[-1] / ipc['i']
ipc.head()

,year,month,i,CPI_Multiplier
0,2016,январь,100.96,0.996434
1,2016,февраль,100.63,0.999702
2,2016,март,100.46,1.001394
3,2016,апрель,100.44,1.001593
4,2016,май,100.41,1.001892


In [24]:
#Замена данных без влияния инфляции

for i in forecast_categories:
    
    #Расчет продаж без влияния инфляции
    frames_dict[i] = frames_dict[i].merge(ipc, how='left', on = ['month','year'] )
    frames_dict[i]['i_sales'] = frames_dict[i]['Sales'] * frames_dict[i]['CPI_Multiplier']
    
    #Замена отсутсвующих значений (нет ИПЦ текущего года) фактическими
    frames_dict[i].loc[frames_dict[i].i_sales.isna(), 'i_sales'] = frames_dict[i].Sales
    

# Корректировка локдаунов

In [25]:
#В 2020 ГОДУ

for i in forecast_categories:
    merge_ipc = frames_dict[i]

    #Данные только 2021 года
    data_2021 = merge_ipc[merge_ipc['year']==2021].reset_index(drop=True)


    #Данные только 2020 года
    data_2020 = merge_ipc[merge_ipc['year']==2020].reset_index(drop=True)


    #Декомпозиция ряда продаж 2021 года
    sales_seasonal_2021 = sm.tsa.seasonal_decompose(data_2021.i_sales, model='additive',period = 4, extrapolate_trend=0)

    #Модель на 2021 год
    forecast_data_2021 = pd.DataFrame({'year': data_2020.year, 'week':data_2020.week,
                                  'trend_sales': sales_seasonal_2021.trend, 'season_sales': sales_seasonal_2021.seasonal})


    forecast_data_2021['forecast_sales'] = forecast_data_2021.trend_sales + forecast_data_2021.season_sales


    #Фактические данные 2020 года
    forecast_data_2021['real_2020_sales'] = data_2020.i_sales.reset_index(drop=True)


    #Разница между фактическими продажами 2020 года и расчетными 2021 года
    forecast_data_2021['diff_sales']=forecast_data_2021.real_2020_sales-forecast_data_2021.forecast_sales
    forecast_data_2021['abs_diff_sales']=abs(forecast_data_2021['diff_sales'])


    #Если разница < чем 30 000 000, то абсолютное значение разницы прибавляем к модельному значению
    forecast_data_2021.loc[forecast_data_2021['abs_diff_sales']>forecast_data_2021['abs_diff_sales'].std(), 
                                                       'real_2020_sales'] = forecast_data_2021.forecast_sales


    #Восстановленный ряд 2020 года:
    forecast_data_2021=forecast_data_2021.drop(['season_sales','trend_sales', 
                                                'diff_sales', 'abs_diff_sales','forecast_sales'], axis=1)


    #Заменяем данные 2020 года на восстановленные в исходном датасете

    saved_2020_data=merge_ipc.merge(forecast_data_2021, how = 'left', on=['year', 'week'])
    saved_2020_data.loc[saved_2020_data.real_2020_sales.notna(), 'i_sales']=saved_2020_data.real_2020_sales
    saved_2020_data = saved_2020_data.drop(['real_2020_sales', 'i','CPI_Multiplier','Sales'], axis=1)
    frames_dict[i] = saved_2020_data 

In [26]:
#В 2021 ГОДУ

for i in forecast_categories:
    saved_2020_data = frames_dict[i]

    #Данные только 2021 года
    data_2021 = saved_2020_data[saved_2020_data['year']==2021].reset_index(drop=True)


    #Данные только 2020 года
    data_2020 = saved_2020_data[saved_2020_data['year']==2020].reset_index(drop=True)


    #Декомпозиция ряда продаж 2021 года
    sales_seasonal_2020 = sm.tsa.seasonal_decompose(data_2020.i_sales, model='additive',period = 4, extrapolate_trend=0)

    #Модель на 2021 год
    forecast_data_2020 = pd.DataFrame({'year': data_2021.year, 'week':data_2021.week,
                                  'trend_sales': sales_seasonal_2020.trend, 'season_sales': sales_seasonal_2020.seasonal})


    forecast_data_2020['forecast_sales'] = forecast_data_2020.trend_sales + forecast_data_2020.season_sales


    #Фактические данные 2021 года
    forecast_data_2020['real_2021_sales'] = data_2021.i_sales.reset_index(drop=True)


    #Если разница недели локдауна, заменяем их на тенденцию
    forecast_data_2020.loc[forecast_data_2020.week.isin(weeks_lockdown_2021),
                                                       'real_2021_sales'] = forecast_data_2020.forecast_sales


    #Восстановленный ряд 2021 года:
    forecast_data_2020=forecast_data_2020.drop(['season_sales','trend_sales','forecast_sales'], axis=1)


    #Заменяем данные 2021 года на восстановленные в исходном датасете
    saved_2021_data=saved_2020_data.merge(forecast_data_2020, how = 'left', on=['year', 'week'])
    saved_2021_data.loc[saved_2021_data.real_2021_sales.notna(), 'i_sales']=saved_2021_data.real_2021_sales
    saved_2021_data = saved_2021_data.drop(['real_2021_sales'], axis=1)
    frames_dict[i]=saved_2021_data

# График восстановленных данных¶

In [27]:
# for i in forecast_categories:
#     plt.title(i)
#     plt.plot(frames_dict[i].i_sales)
#     plt.show()

# Скользящая переменная для прогноза

In [28]:
#Берем среднее за 2 прошедшие недели

for i in forecast_categories:
 

    mean_s = frames_dict[i].i_sales.rolling(window=2).mean().dropna()
    auto_data = pd.DataFrame({'i_sales' : frames_dict[i].i_sales[1:].reset_index(drop=True), 
                              'week' : frames_dict[i].week[1:].reset_index(drop=True),
                              'year' : frames_dict[i].year[1:].reset_index(drop=True),
                              'mean_s': mean_s.reset_index(drop=True)
                 })


    frames_dict[i] = auto_data

# Обучение

In [29]:
copy_dict = frames_dict.copy()
forecast_dict = {}

In [30]:
%%time

for ctg in forecast_categories:
    data_for_forecast = frames_dict[ctg]


    #Формируем обучающий датасет

    target = data_for_forecast['i_sales']
    lean_df = data_for_forecast.drop(['i_sales'], axis=1)


    #Дамми-переменные из недель
    dummyes = pd.get_dummies(lean_df.week)
    dummyes['week']=lean_df.week
    dummyes['year']=lean_df.year
    lean_df=lean_df.merge(dummyes, how = 'left', on=['week', 'year'])

    #Делим обучающие датасеты на общий и сезонный
    lean_df_model = lean_df.drop(['week', 'year'], axis=1)
    lean_df_season = lean_df.drop(['week', 'year','mean_s'], axis=1)


#     #тест и трейн для общей модели
#     y_train, y_test, X_train, X_test = train_test_split(target, lean_df_model,
#                                     shuffle = False,
#                                     test_size=test_size, 
#                                     random_state=0)


#     #Тест и трейн для сезонной модели
#     y_tr, y_tst, X_tr, X_tst = train_test_split(target, lean_df_season,
#                                     shuffle = False,
#                                     test_size=test_size, 
#                                     random_state=0)

    #Сезонная модель
    reg_season = RandomForestRegressor(bootstrap=True, n_estimators=2000)
    reg_season.fit(lean_df_season, target)
    season_pred = pd.DataFrame(reg_season.predict(lean_df_season))

    #Расчет поправочного коэффициента
    season_pred['k']=season_pred.values/season_pred.iloc[[0]].values


    #ОБЩАЯ МОДЕЛЬ
    reg = RandomForestRegressor(bootstrap=True, n_estimators=2000)
    reg.fit(lean_df_model, target)
    train_predictions = reg.predict(lean_df_model)


    #Пересчет X_test (база для прогноза)
    
    X_test = pd.DataFrame({'week':[], 'year':[], 'mean_s':[] })
    X_test['week']= list(range(week_number, week_number+horizon))
    X_test['year'] = year_number
    X_test.loc[X_test.week>52, 'year'] = X_test.year+1
    X_test.loc[X_test.week>52, 'week'] = X_test.week-52
    
    #Дамми-переменные из недель X_test
    X_test = X_test[['week','year','mean_s']].merge(dummyes.iloc[:,:-1], how = 'left', on = 'week').drop_duplicates()
    forecast_weeks = X_test.week
    forecast_year = X_test.year
    X_test = X_test.drop(['week', 'year'], axis=1)

    #Среднее скользящее в прогнозе
    X_test.mean_s[0] = (target.iloc[-2]+target.iloc[-1])/2
    X_index = list(range(list(lean_df.index)[-1]+1, list(lean_df.index)[-1]+1+horizon))
    X_test.index = X_index
    forecast_weeks.index = X_index
    forecast_year.index = X_index
    forecast_index = X_test.index[1:]
    X_test.mean_s[forecast_index[0]]=(target.iloc[-1]+train_predictions[-1:])/2


#     for i in forecast_index[1:]:
#         X_test.mean_s[i]=(reg.predict(X_test.loc[[i-2]]) + reg.predict(
#                             X_test.loc[[i-1]])+season_pred.iloc[[i-52],0].values)/3


    for i in forecast_index[1:]:
        X_test.mean_s[i]=(reg.predict(X_test.loc[[i-2]]) + reg.predict(
                            X_test.loc[[i-1]]) )/2     
    
    #Коэффициент для корректировки прогноза
    k = season_pred.loc[list(X_test.index-52)]
    k.index = X_test.index 
    k['k']=k.values/k.iloc[[0]].values

    #Прогноз с учетом поправочного коэффициента
    test_predictions = reg.predict(X_test)
    test_predictions[1:]=test_predictions[1:]*k.k[1:]

    forecast_dict[ctg] = test_predictions
    
    #Индексация для графиков
    test_index = X_test.index
    forecast_index = target.index[-1]


    # #график общий
    # fig, ax = plt.subplots(figsize=(15,5),dpi=500)
    # ax.set_xticklabels(full_labels)
    # plt.plot(train_predictions, label='training')
    # plt.plot(target, label='actual')
    # plt.plot( test_index, test_predictions, label='forecast')
    # plt.title('RandomForestRegressor')
    # plt.legend(loc='upper left', fontsize=8)
    # plt.show()


    #график валидационный
#     fig, ax = plt.subplots(figsize=(15,5),dpi=500)
# #     plt.plot(y_test.reset_index(drop=True), label='actual')
#     plt.plot(test_predictions,label='forecast')
#     plt.title(ctg)
#     plt.legend(loc='upper left', fontsize=8)
#     plt.show()

Wall time: 8min 55s


In [31]:
#Конкатенация всех прогнозов

final_forecast_data = pd.DataFrame(forecast_dict, index = test_index)
d2 = pd.DataFrame({'forecast':[], 'uni':[], 'year':[], 'week':[]})

for i in forecast_categories:   
    d = pd.DataFrame(final_forecast_data[i])
    d = d.rename(columns={i: "forecast"})
    d['uni'] = i
    d['year'] = forecast_year
    d['week'] = forecast_weeks
    d2 = pd.concat([d,d2])

d2['week'] = d2.week.astype(int)
d2['year'] = d2.year.astype(int)

# Обработка датафрейма перед выгрузкой

In [41]:
dwld_data = d2.merge(data_for_final[['TG', 'ТГ+', 'uni']].drop_duplicates(), how = 'inner', on = 'uni' )
dwld_data = dwld_data[['year','week','TG', 'ТГ+','forecast']]

#Прогноз зашумлен, формула зашумления удалена (прогноз != истинному прогнозу)
dwld_data

,year,week,TG,ТГ+,forecast
0,2021,47,B1-8,SHIRTS L-SL,2.649409e+06
1,2021,48,B1-8,SHIRTS L-SL,2.636326e+06
2,2021,49,B1-8,SHIRTS L-SL,2.654721e+06
3,2021,50,B1-8,SHIRTS L-SL,2.741628e+06
4,2021,51,B1-8,SHIRTS L-SL,2.767212e+06
...,...,...,...,...,...
763,2022,2,G13+,JEANS COMMERCIAL,3.303427e+06
764,2022,3,G13+,JEANS COMMERCIAL,3.247363e+06
765,2022,4,G13+,JEANS COMMERCIAL,3.244602e+06
766,2022,5,G13+,JEANS COMMERCIAL,3.263939e+06


# Выгрузка датафрейма

In [34]:
dwld_data.to_excel('forecast.xlsx', index = False)